In [ ]:
!pip install Unidecode
!pip install keybert

!pip install pysentimiento
!pip install accelerate>=0.20.1
!pip install transformers[torch]

In [3]:
# Prevenir grandes mensajes de advertencia de BertModel
import warnings
warnings.filterwarnings("ignore") 

# Importar paquetes
import pandas as pd
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm.pandas()

import sys
import unidecode
import re
import string

with open('openai-key.txt') as f:
    API_KEY = f.readlines()
    
# Leer archivo CSV
df = pd.read_csv("data/noticias-losrios.csv")[:100]
df.drop(df[df['text'].str.len() < 10].index, inplace=True)
df.drop(columns=["Unnamed: 0"], inplace=True)
df['content'] = df['title'] + "  " + df['text']
print(len(df))
df.head()

100


,id_news,country,media_outlet,url,title,text,date,content
0,47314428.0,chile,suractual,https://www.suractual.cl/2022/01/24/ante-proye...,Ante proyecto Tierras Raras municipio y Unión ...,“Considerando la evolución que ha tenido el Pr...,2022-01-24,Ante proyecto Tierras Raras municipio y Unión ...
1,47719086.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/25/trac...,Tractocamión volcó esta mañana en puente Lloll...,El accidente se produjo pasadas las 06:00 AM y...,2022-01-25,Tractocamión volcó esta mañana en puente Lloll...
2,47719110.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/27/gobe...,Gobernadores Regionales de Los Ríos y Los Lago...,La instancia buscó generar un trabajo colabora...,2022-01-27,Gobernadores Regionales de Los Ríos y Los Lago...
3,47700339.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Alegan inocencia de acusadas por cruel asesina...,La defensa de las dos mujeres acusadas de enca...,2022-01-05,Alegan inocencia de acusadas por cruel asesina...
4,47700425.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Destacan ejecución del Programa Vínculos en Lo...,Para dar protección y acompañamiento a adultos...,2022-01-03,Destacan ejecución del Programa Vínculos en Lo...


<img src="https://www.goredelosrios.cl/wp-content/uploads/urai/maparegiondelosrios.png"/>

In [4]:
def clean_text(text):
    # transformar caracteres acentuados a no acentuados
    text = unidecode.unidecode(str(text))
    # borrar caracteres distintos a los alfabeticos
    text = re.sub(r'[^A-Za-z ]+', "", text) 
    # borrar puntuaciones
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # borrar palabras comprometidas con numeros
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text)
    
    return text

def find_cities(text):
    cities = [
        "mariquina", "lanco", "mafil", "valdivia", "corral",
        "paillaco", "los lagos", "panguipulli", "la unión",
        "rio bueno", "lago ranco", "futrono"
    ]
    text = clean_text(text)
    found = [city for city in cities if city in text] if text is not None else []
    
    return found


In [5]:
# Encontrar ciudades mencionadas en el contenido de noticias
df['cities'] = df.content.progress_apply(lambda doc: find_cities(doc))

# Obtener la lista de documentos
docs = df.content.tolist()

# Convertir la columna 'date' a formato de fecha
df['date'] = pd.to_datetime(df['date'])

# Imprimir el número de noticias procesadas
print("Number of news:", len(df))

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 578.05it/s]


Number of news: 100


In [6]:
city_counts = df['cities'].value_counts()

city_dict = {}
for cities, count in city_counts.items():
    for city in cities:
        if city in city_dict:
            city_dict[city] += count
        else:
            city_dict[city] = count

city_dict

{'valdivia': 12, 'panguipulli': 1, 'lanco': 1}

In [10]:
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT

kw_model = KeyBERT(model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2"))
df['keywords'] = df.content.progress_apply(lambda doc: kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3),
                              use_mmr=True, diversity=0.7))


100%|█████████████████████████████████████████| 100/100 [02:49<00:00,  1.69s/it]


In [32]:
import re
for index, row in df.iterrows():
    keywords = []

    for item in row['keywords']:
        match = re.search(r"'([^']*)'", str(row['keywords']))
        if match:
            keywords.append(match.group(1))
    df.at[index, 'keywords'] = keywords


In [34]:
from pysentimiento import create_analyzer
sentiment_analyzer = create_analyzer(task="sentiment", lang="es")

In [35]:
def sentiment_analysis_news(df, sentiment_analyzer):
    
    for index, row in tqdm(df.iterrows(), desc='rows - sentiment', total=df.shape[0]):
        # análisis del título de la noticia
        label = sentiment_analyzer.predict(row['title']).output
        score = list(sentiment_analyzer.predict(row['title']).probas.values())[0]
        df.at[index, "label_roBERTuito"] = label
        df.at[index, "score_roBERTuito"] = score
    return df

In [36]:
df = sentiment_analysis_news(df, sentiment_analyzer)

rows - sentiment: 100%|███████████████████████| 100/100 [00:08<00:00, 11.28it/s]


In [37]:
df

,id_news,country,media_outlet,url,title,text,date,content,cities,keywords,label_roBERTuito,score_roBERTuito
0,47314428.0,chile,suractual,https://www.suractual.cl/2022/01/24/ante-proye...,Ante proyecto Tierras Raras municipio y Unión ...,“Considerando la evolución que ha tenido el Pr...,2022-01-24,Ante proyecto Tierras Raras municipio y Unión ...,[],"[consulta ciudadana comunal, consulta ciudadan...",NEU,0.072054
1,47719086.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/25/trac...,Tractocamión volcó esta mañana en puente Lloll...,El accidente se produjo pasadas las 06:00 AM y...,2022-01-25,Tractocamión volcó esta mañana en puente Lloll...,[],"[puente llollelhue, puente llollelhue, puente ...",NEU,0.371151
2,47719110.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/27/gobe...,Gobernadores Regionales de Los Ríos y Los Lago...,La instancia buscó generar un trabajo colabora...,2022-01-27,Gobernadores Regionales de Los Ríos y Los Lago...,[],"[gobierno regional los, gobierno regional los,...",NEU,0.046309
3,47700339.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Alegan inocencia de acusadas por cruel asesina...,La defensa de las dos mujeres acusadas de enca...,2022-01-05,Alegan inocencia de acusadas por cruel asesina...,[],"[los acusados homicidio, los acusados homicidi...",NEG,0.509495
4,47700425.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Destacan ejecución del Programa Vínculos en Lo...,Para dar protección y acompañamiento a adultos...,2022-01-03,Destacan ejecución del Programa Vínculos en Lo...,[],"[la emergencia sanitaria, la emergencia sanita...",NEU,0.058263
...,...,...,...,...,...,...,...,...,...,...,...,...
95,47689820.0,chile,diariodevaldivia,https://www.diariodevaldivia.cl/noticia/emerge...,Conductor con alcohol protagonizó violento acc...,Diez personas lesionadas es el saldo de un vio...,2022-01-20,Conductor con alcohol protagonizó violento acc...,[],"[violento accidente en, violento accidente en,...",NEG,0.705443
96,47689821.0,chile,diariodevaldivia,https://www.diariodevaldivia.cl/noticia/actual...,Carabinero salva a persona que se lanzó desde ...,Un carabinero protagonizó el rescate de un jov...,2022-01-19,Carabinero salva a persona que se lanzó desde ...,[],"[carabinero salva persona, carabinero salva pe...",NEU,0.327833
97,47689822.0,chile,diariodevaldivia,https://www.diariodevaldivia.cl/noticia/emerge...,"La oscura trama de drogas, dinero, venganza y ...",Mayo 2020.- Con una serie de datos claves prop...,2022-01-19,"La oscura trama de drogas, dinero, venganza y ...",[],"[crimendehelena el hallazgo, crimendehelena el...",NEG,0.729149
98,47689823.0,chile,diariodevaldivia,https://www.diariodevaldivia.cl/noticia/actual...,Más tormentas eléctricas anunciadas para los c...,De acuerdo con la información proporcionada po...,2022-01-18,Más tormentas eléctricas anunciadas para los c...,[],"[tormentas eléctricas anunciadas, tormentas el...",NEU,0.341036


In [39]:
#guardar en un CSV
df.to_csv("data/noticias-etiquetadas.csv")